### Installation and importing client ids

In [ ]:
!pip install spotipy

To install torch: conda install pytorch torchvision torchaudio cpuonly -c pytorch

Also need to install: matplotlib and pandas

In [ ]:
import os
import requests
import torch
import torchaudio
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [73]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
cid = '49789426e6c04428a2befbd3c2d79b02'
secret = 'd18032252a0d4634aa8ca21356894e79'
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

#### Import the existing csv files as dataframes

In [ ]:
danceable_tracks_df = pd.read_csv('/Users/cheffbcookin/Desktop/EC523/Song_Feature_Pred/Sample_Data/danceable_tracks-1.csv')
non_danceable_tracks_df = pd.read_csv('/Users/cheffbcookin/Desktop/EC523/Song_Feature_Pred/Sample_Data/non-danceable_tracks-1.csv')
instrumental_tracks_df = pd.read_csv('/Users/cheffbcookin/Desktop/EC523/Song_Feature_Pred/Sample_Data/instrumental_tracks-1.csv')
non_instrumental_tracks_df = pd.read_csv('/Users/cheffbcookin/Desktop/EC523/Song_Feature_Pred/Sample_Data/non-instrumental_tracks-1.csv')
speechy_tracks_df = pd.read_csv('/Users/cheffbcookin/Desktop/EC523/Song_Feature_Pred/Sample_Data/speechy_tracks-1.csv')
non_speechy_tracks_df = pd.read_csv('/Users/cheffbcookin/Desktop/EC523/Song_Feature_Pred/Sample_Data/non-speechy_tracks-1.csv')
acoustic_tracks_df = pd.read_csv('/Users/cheffbcookin/Desktop/EC523/Song_Feature_Pred/Sample_Data/acoustic_tracks-1.csv')
non_acoustic_tracks_df = pd.read_csv('/Users/cheffbcookin/Desktop/EC523/Song_Feature_Pred/Sample_Data/non-acoustic_tracks-1.csv')
energetic_tracks_df = pd.read_csv('/Users/cheffbcookin/Desktop/EC523/Song_Feature_Pred/Sample_Data/energetic_tracks-1.csv')
non_energetic_tracks_df = pd.read_csv('/Users/cheffbcookin/Desktop/EC523/Song_Feature_Pred/Sample_Data/non-energetic_tracks-1.csv')
all_tracks_df = pd.read_csv('/Users/cheffbcookin/Desktop/EC523/Song_Feature_Pred/Sample_Data/all_tracks-1.csv')

In [ ]:
dataframes = {
    'Danceable Tracks': danceable_tracks_df,
    'Non-danceable Tracks': non_danceable_tracks_df,
    'Instrumental Tracks': instrumental_tracks_df,
    'Non-instrumental Tracks': non_instrumental_tracks_df,
    'Speechy Tracks': speechy_tracks_df,
    'Non-speechy Tracks': non_speechy_tracks_df,
    'Acoustic Tracks': acoustic_tracks_df,
    'Non-acoustic Tracks': non_acoustic_tracks_df,
    'Energetic Tracks': energetic_tracks_df,
    'Non-energetic Tracks': non_energetic_tracks_df,
    'All Tracks': all_tracks_df
}

In [ ]:
for dataframe in dataframes:
    print(dataframe)
    print(dataframes[dataframe].head(10))
    print(dataframes[dataframe].columns)
    print()

##### Initialize dictionary to hold list of tracks with specific features

We will later save this dictionary and reuse it 

In [ ]:
# Load existing DataFrame from CSV file (if it exists)
try:
    tracks_data_base = pd.read_csv('tracks_data.csv')
except FileNotFoundError:
    tracks_data_base = pd.DataFrame(columns=['track_id', 'track_name', 'track_artist', 'preview_url'])
    
# Load existing DataFrame from CSV file (if it exists)
try:
    tracks_data = pd.read_csv('tracks_data_plus.csv')
except FileNotFoundError:
    tracks_data = pd.DataFrame(columns=['track_id', 'track_name', 'track_artist', 'preview_url'])

Experimenting with grabbing data using just the feature and values

In [ ]:
# Define danceability range and step size
start_danceability = 0.7
end_danceability = 1.0
step_size = 0.001

# Iterate over danceability values and perform searches
for danceability in reversed(range(int(start_danceability / step_size), int(end_danceability / step_size))):
    # Define search query
    query = f'danceability:{danceability*step_size:.2f}'

    # Search for tracks
    results = sp.search(q=query, type='track', market='US', limit=50)

    # Add new tracks to tracks_data DataFrame (if not already in DataFrame)
    for track in results['tracks']['items']:
        if track['id'] is not None and track['id'] not in tracks_data['track_id'].values:
            track_name = track['name']
            track_artist = track['artists'][0]['name']
            preview_url = track['preview_url']
            tracks_data = tracks_data.append({'track_id': track_id, 'track_name': track_name, 'track_artist': track_artist, 'preview_url': preview_url}, ignore_index=True)

# Print number of tracks found and write to CSV file
print(f'Found {len(tracks_data)} tracks')

# Print tracks_data DataFrame
print(tracks_data.head())

Reload the state of the large audio dataframes

In [ ]:
# We will save the data to a CSV file so that we can use it later

#tracks_data_base.to_csv('tracks_data.csv', index=False)
tracks_data_base = pd.read_csv('tracks_data.csv')
print(f'Found {len(tracks_data_base)} tracks')

In [ ]:
# load the data from the CSV file
tracks_data = pd.read_csv('tracks_data_plus.csv')
print(f'Found {len(tracks_data)} tracks')

In [ ]:
# Define feature range and step size
start = 0.0
end = 1.0
step_size = 0.005

# Iterate over danceability values and perform searches
for feature in reversed(range(int(start / step_size), int(end / step_size))):
    # Define search query
    print(f'working with feature interval: {feature*step_size:.3f}')
    query = f'danceability:{feature*step_size:.3f}'

    # Search for tracks
    results = sp.search(q=query, type='track', market='US', limit=50)

    # Add new tracks to tracks_data DataFrame (if not already in DataFrame)
    for track in results['tracks']['items']:
        if track['id']:
            if track['id'] is not None and track['id'] not in tracks_data['track_id'].values:
                track_name = track['name']
                track_artist = track['artists'][0]['name']
                preview_url = track['preview_url']
                tracks_data = tracks_data.append({'track_id': track_id, 'track_name': track_name, 'track_artist': track_artist, 'preview_url': preview_url}, ignore_index=True)

    # Print number of tracks found and write to CSV file
    print(f'Found {len(tracks_data)} tracks')
    tracks_data.to_csv('tracks_data.csv', index=False)
    
    # Define search query
    query = f'instrumentalness:{feature*step_size:.3f}'

    # Search for tracks
    results = sp.search(q=query, type='track', market='US', limit=50)

    # Add new tracks to tracks_data DataFrame (if not already in DataFrame)
    for track in results['tracks']['items']:
        if track['id']:
            track_id = track['id']
            if track_id not in tracks_data['track_id'].values:
                track_name = track['name']
                track_artist = track['artists'][0]['name']
                preview_url = track['preview_url']
                tracks_data = tracks_data.append({'track_id': track_id, 'track_name': track_name, 'track_artist': track_artist, 'preview_url': preview_url}, ignore_index=True)

    # Print number of tracks found and write to CSV file
    print(f'Found {len(tracks_data)} tracks')
    tracks_data.to_csv('tracks_data.csv', index=False)
    
    # Define search query
    query = f'speechiness:{feature*step_size:.3f}'

    # Search for tracks
    results = sp.search(q=query, type='track', market='US', limit=50)

    # Add new tracks to tracks_data DataFrame (if not already in DataFrame)
    for track in results['tracks']['items']:
        if track['id']:
            if track['id'] is not None and track['id'] not in tracks_data['track_id'].values:
                track_name = track['name']
                track_artist = track['artists'][0]['name']
                preview_url = track['preview_url']
                tracks_data = tracks_data.append({'track_id': track_id, 'track_name': track_name, 'track_artist': track_artist, 'preview_url': preview_url}, ignore_index=True)

    # Print number of tracks found and write to CSV file
    print(f'Found {len(tracks_data)} tracks')
    tracks_data.to_csv('tracks_data.csv', index=False)
    
    # Define search query
    query = f'acousticness:{feature*step_size:.3f}'

    # Search for tracks
    results = sp.search(q=query, type='track', market='US', limit=50)

    # Add new tracks to tracks_data DataFrame (if not already in DataFrame)
    for track in results['tracks']['items']:
        if track['id']:
            if track['id'] is not None and track['id'] not in tracks_data['track_id'].values:
                track_name = track['name']
                track_artist = track['artists'][0]['name']
                preview_url = track['preview_url']
                tracks_data = tracks_data.append({'track_id': track_id, 'track_name': track_name, 'track_artist': track_artist, 'preview_url': preview_url}, ignore_index=True)

    # Print number of tracks found and write to CSV file
    print(f'Found {len(tracks_data)} tracks')
    tracks_data.to_csv('tracks_data.csv', index=False)
    
    # Define search query
    query = f'energy:{feature*step_size:.3f}'

    # Search for tracks
    results = sp.search(q=query, type='track', market='US', limit=50)

    # Add new tracks to tracks_data DataFrame (if not already in DataFrame)
    for track in results['tracks']['items']:
        if track['id']:
            if track['id'] is not None and track['id'] not in tracks_data['track_id'].values:
                track_name = track['name']
                track_artist = track['artists'][0]['name']
                preview_url = track['preview_url']
                tracks_data = tracks_data.append({'track_id': track_id, 'track_name': track_name, 'track_artist': track_artist, 'preview_url': preview_url}, ignore_index=True)

    # Print number of tracks found and write to CSV file
    print(f'Found {len(tracks_data)} tracks')
    tracks_data.to_csv('tracks_data.csv', index=False)

#### THIS IS ONLY IF WE NEED EVEN MORE DATA

We did indeed need even more metadata

In [ ]:
# Define feature range and step size
start = 0.0
end = 1.0
step_size = 0.001

# Iterate over danceability values and perform searches
for feature in reversed(range(int(start / step_size), int(end / step_size))):
    # Define search query
    print(f'working with feature interval: {feature*step_size:.3f}')
    query = f'danceability:{feature*step_size:.3f}'

    # Search for tracks
    results = sp.search(q=query, type='track', market='US', limit=50)

    # Add new tracks to tracks_data DataFrame (if not already in DataFrame)
    for track in results['tracks']['items']:
        if track['id'] is not None and track['id'] not in tracks_data['track_id'].values:
            track_name = track['name']
            track_artist = track['artists'][0]['name']
            preview_url = track['preview_url']
            df_temp = pd.DataFrame({'track_id': track_id, 'track_name': track_name, 'track_artist': track_artist, 'preview_url': preview_url}, index=[0])
            #tracks_data = tracks_data.append({'track_id': track_id, 'track_name': track_name, 'track_artist': track_artist, 'preview_url': preview_url}, ignore_index=True)
            tracks_data = pd.concat([tracks_data, df_temp], axis=0)
            
    # Print number of tracks found and write to CSV file
    print(f'Found {len(tracks_data)} tracks')
    tracks_data.to_csv('tracks_data_plus.csv', index=False)
    
    # Define search query
    query = f'instrumentalness:{feature*step_size:.3f}'

    # Search for tracks
    results = sp.search(q=query, type='track', market='US', limit=50)

    # Add new tracks to tracks_data DataFrame (if not already in DataFrame)
    for track in results['tracks']['items']:
        if track['id'] is not None and track['id'] not in tracks_data['track_id'].values:
            track_name = track['name']
            track_artist = track['artists'][0]['name']
            preview_url = track['preview_url']
            df_temp = pd.DataFrame({'track_id': track_id, 'track_name': track_name, 'track_artist': track_artist, 'preview_url': preview_url}, index=[0])
            #tracks_data = tracks_data.append({'track_id': track_id, 'track_name': track_name, 'track_artist': track_artist, 'preview_url': preview_url}, ignore_index=True)
            tracks_data = pd.concat([tracks_data, df_temp], axis=0)
    # Print number of tracks found and write to CSV file
    print(f'Found {len(tracks_data)} tracks')
    tracks_data.to_csv('tracks_data_plus.csv', index=False)
    
    # Define search query
    query = f'speechiness:{feature*step_size:.3f}'

    # Search for tracks
    results = sp.search(q=query, type='track', market='US', limit=50)

    # Add new tracks to tracks_data DataFrame (if not already in DataFrame)
    for track in results['tracks']['items']:
        if track['id'] is not None and track['id'] not in tracks_data['track_id'].values:
            track_name = track['name']
            track_artist = track['artists'][0]['name']
            preview_url = track['preview_url']
            df_temp = pd.DataFrame({'track_id': track_id, 'track_name': track_name, 'track_artist': track_artist, 'preview_url': preview_url}, index=[0])
            #tracks_data = tracks_data.append({'track_id': track_id, 'track_name': track_name, 'track_artist': track_artist, 'preview_url': preview_url}, ignore_index=True)
            tracks_data = pd.concat([tracks_data, df_temp], axis=0)
    # Print number of tracks found and write to CSV file
    print(f'Found {len(tracks_data)} tracks')
    tracks_data.to_csv('tracks_data_plus.csv', index=False)
    
    # Define search query
    query = f'acousticness:{feature*step_size:.3f}'

    # Search for tracks
    results = sp.search(q=query, type='track', market='US', limit=50)

    # Add new tracks to tracks_data DataFrame (if not already in DataFrame)
    for track in results['tracks']['items']:
        if track['id'] is not None and track['id'] not in tracks_data['track_id'].values:
            track_name = track['name']
            track_artist = track['artists'][0]['name']
            preview_url = track['preview_url']
            df_temp = pd.DataFrame({'track_id': track_id, 'track_name': track_name, 'track_artist': track_artist, 'preview_url': preview_url}, index=[0])
            #tracks_data = tracks_data.append({'track_id': track_id, 'track_name': track_name, 'track_artist': track_artist, 'preview_url': preview_url}, ignore_index=True)
            tracks_data = pd.concat([tracks_data, df_temp], axis=0)
    # Print number of tracks found and write to CSV file
    print(f'Found {len(tracks_data)} tracks')
    tracks_data.to_csv('tracks_data_plus.csv', index=False)
    
    # Define search query
    query = f'energy:{feature*step_size:.3f}'

    # Search for tracks
    results = sp.search(q=query, type='track', market='US', limit=50)

    # Add new tracks to tracks_data DataFrame (if not already in DataFrame)
    for track in results['tracks']['items']:
        if track['id'] is not None and track['id'] not in tracks_data['track_id'].values:
            track_name = track['name']
            track_artist = track['artists'][0]['name']
            preview_url = track['preview_url']
            df_temp = pd.DataFrame({'track_id': track_id, 'track_name': track_name, 'track_artist': track_artist, 'preview_url': preview_url}, index=[0])
            #tracks_data = tracks_data.append({'track_id': track_id, 'track_name': track_name, 'track_artist': track_artist, 'preview_url': preview_url}, ignore_index=True)
            tracks_data = pd.concat([tracks_data, df_temp], axis=0)
            
    # Print number of tracks found and write to CSV file
    print(f'Found {len(tracks_data)} tracks')
    tracks_data.to_csv('tracks_data_plus.csv', index=False)

#### Label collection

We've done the grunt work to collect all the tracks that would provide us with a wide range of data for our desired features. Now we go through and grab all the audio features for this set of tracks

In [ ]:
# Step 1: Obtain an access token
auth_url = 'https://accounts.spotify.com/api/token'

auth_response = requests.post(
    auth_url,
    data={
        'grant_type': 'client_credentials',
        'client_id': cid,
        'client_secret': secret
    }
)

auth_response_data = auth_response.json()
access_token = auth_response_data['access_token']

print(access_token)

headers = {
    'Authorization': f'Bearer {access_token}'
}

In [ ]:
tracks_data_base = pd.read_csv('tracks_data.csv')
print(f'Found {len(tracks_data_base)} tracks')

tracks_data = pd.read_csv('tracks_data_plus.csv')
print(f'Found {len(tracks_data)} tracks')

In [ ]:
# Set a threshold for desired features
danceability_threshold = 0.5
non_danceability_threshold = 0.5

instrumentalness_threshold = 0.5
non_instrumentalness_threshold = 0.5

speechiness_threshold = 0.5
non_speechiness_threshold = 0.5

acousticness_threshold = 0.5
non_acousticness_threshold = 0.5

energy_threshold = 0.5
non_energy_threshold = 0.5

total_danceable = 0
total_non_danceable = 0

total_instrumental = 0
total_non_instrumental = 0

total_speechy = 0
total_non_speechy = 0

total_acoustic = 0
total_non_acoustic = 0

total_energetic = 0
total_non_energetic = 0

total = 0


for index, track in tracks_data.iterrows():
    # k += 1
    track_id = track['track_id']
    track_name = track['track_name']
    track_artist = track['track_artist']
    preview_url = track['preview_url']
    
    if preview_url and track_id not in all_tracks_df['track_id'].values:
        # Fetch track's audio features
        audio_features_url = f'https://api.spotify.com/v1/audio-features/{track_id}'
        audio_features_response = requests.get(audio_features_url, headers=headers)
        audio_features_data = audio_features_response.json()
        
        # Check if all audio features are present
        required_features = ['danceability', 'instrumentalness', 'speechiness', 'acousticness', 'energy', 'valence', 'tempo', 'loudness', 'liveness', 'key', 'mode', 'time_signature']
        if not all(feature in audio_features_data for feature in required_features):
            # Remove the current row if any feature is missing
            print(f'removing track {index+1} of {len(tracks_data)} from large numeric_based dataset')
            tracks_data = tracks_data.drop(index)
            
            # check the necessary labels and remove tracks if the bare minimum is not met:
            bare_minimum_required_labels = ['instrumentalness', 'speechiness', 'acousticness', 'energy']
            if not all(feature in audio_features_data for feature in bare_minimum_required_labels):
                
                track_id_to_remove = track_id

                # Find the index of the row with the track_id_to_remove value
                if track_id_to_remove in tracks_data_base['track_id'].values:
                    index_to_remove = tracks_data_base[tracks_data_base['track_id'] == track_id_to_remove].index
                    # Remove rows with track ids in the list
                    # List of track ids to remove
                    if index_to_remove.empty:
                        print(f'Track {track_id_to_remove} not found in tracks_data_base.')
                    else:
                        print(f'removing track from smaller dataset of length: {len(tracks_data_base)}')
                        tracks_data_base = tracks_data_base.drop(index_to_remove)
                        # Reindex the dataframe to start from 0
                        tracks_data_base = tracks_data_base.reset_index(drop=True)
            
            # seeing how danceability is a new feature, we will continue with our labels without it for audio based RNNs
            else:
                print(f'working on track {index+1} of {len(tracks_data)} for smaller dataset')
                # Check if the track is already in the DataFrame
                if preview_url and track_id not in instrumental_tracks_df['track_id'].values:
                
                    # # Fetch track's audio features
                    # audio_features_url = f'https://api.spotify.com/v1/audio-features/{track_id}'
                    # audio_features_response = requests.get(audio_features_url, headers=headers)
                    # audio_features_data = audio_features_response.json()

                    # Check if the track's instrumentalness is greater than the threshold
                    if audio_features_data['instrumentalness'] > instrumentalness_threshold:
                        print(f'\t{track_name} - {track_artist} (Instrumentalness: {audio_features_data["instrumentalness"]})')
                        # Append the new track to the DataFrame
                        track_df = pd.DataFrame({
                            'track_id': [track_id],
                            'track_name': [track_name],
                            'track_artist': [track_artist],
                            'instrumentalness': [audio_features_data['instrumentalness']],
                            'preview_url' : [preview_url]
                        })
                        
                        # Concatenate the new DataFrame with the existing DataFrame
                        instrumental_tracks_df = pd.concat([instrumental_tracks_df, track_df], ignore_index=True)
                        
                        total_instrumental = total_instrumental + 1
                        
                # Check if the track is already in the DataFrame
                if preview_url and track_id not in non_instrumental_tracks_df['track_id'].values:
                
                    # # Fetch track's audio features
                    # audio_features_url = f'https://api.spotify.com/v1/audio-features/{track_id}'
                    # audio_features_response = requests.get(audio_features_url, headers=headers)
                    # audio_features_data = audio_features_response.json()

                    # Check if the track's instrumentalness is less than the threshold
                    if audio_features_data['instrumentalness'] < non_instrumentalness_threshold:
                        print(f'\t{track_name} - {track_artist} (Instrumentalness: {audio_features_data["instrumentalness"]})')
                        # Append the new track to the DataFrame
                        track_df = pd.DataFrame({
                            'track_id': [track_id],
                            'track_name': [track_name],
                            'track_artist': [track_artist],
                            'instrumentalness': [audio_features_data['instrumentalness']],
                            'preview_url' : [preview_url]
                        })
                        
                        # Concatenate the new DataFrame with the existing DataFrame
                        non_instrumental_tracks_df = pd.concat([instrumental_tracks_df, track_df], ignore_index=True)
                        
                        total_non_instrumental = total_non_instrumental + 1
                        
                # Check if the track is already in the DataFrame
                if preview_url and track_id not in speechy_tracks_df['track_id'].values:
                
                    # # Fetch track's audio features
                    # audio_features_url = f'https://api.spotify.com/v1/audio-features/{track_id}'
                    # audio_features_response = requests.get(audio_features_url, headers=headers)
                    # audio_features_data = audio_features_response.json()

                    # Check if the track's speechiness is greater than the threshold
                    if audio_features_data['speechiness'] > speechiness_threshold:
                        print(f'\t{track_name} - {track_artist} (Speechiness: {audio_features_data["speechiness"]})')
                        # Append the new track to the DataFrame
                        track_df = pd.DataFrame({
                            'track_id': [track_id],
                            'track_name': [track_name],
                            'track_artist': [track_artist],
                            'speechiness': [audio_features_data['speechiness']],
                            'preview_url' : [preview_url]
                        })
                        
                        # Concatenate the new DataFrame with the existing DataFrame
                        speechy_tracks_df = pd.concat([speechy_tracks_df, track_df], ignore_index=True)
                        
                        total_speechy = total_speechy + 1
                        
                if preview_url and track_id not in non_speechy_tracks_df['track_id'].values:
                    
                    # # Fetch track's audio features
                    # audio_features_url = f'https://api.spotify.com/v1/audio-features/{track_id}'
                    # audio_features_response = requests.get(audio_features_url, headers=headers)
                    # audio_features_data = audio_features_response.json()
                    
                    # Check if the track's speechiness is less than the threshold
                    if audio_features_data['speechiness'] < non_speechiness_threshold:
                        print(f'\t{track_name} - {track_artist} (Speechiness: {audio_features_data["speechiness"]})')
                        # Append the new track to the DataFrame
                        track_df = pd.DataFrame({
                            'track_id': [track_id],
                            'track_name': [track_name],
                            'track_artist': [track_artist],
                            'speechiness': [audio_features_data['speechiness']],
                            'preview_url' : [preview_url]
                        })
                        
                        # Concatenate the new DataFrame with the existing DataFrame
                        non_speechy_tracks_df = pd.concat([non_speechy_tracks_df, track_df], ignore_index=True)
                        
                        total_non_speechy = total_non_speechy + 1
                        
                if preview_url and track_id not in acoustic_tracks_df['track_id'].values:
                    
                    # # Fetch track's audio features
                    # audio_features_url = f'https://api.spotify.com/v1/audio-features/{track_id}'
                    # audio_features_response = requests.get(audio_features_url, headers=headers)
                    # audio_features_data = audio_features_response.json()
                    
                    # Check if the track's acousticness is greater than the threshold
                    if audio_features_data['acousticness'] > acousticness_threshold:
                        print(f'\t{track_name} - {track_artist} (Acousticness: {audio_features_data["acousticness"]})')
                        # Append the new track to the DataFrame
                        track_df = pd.DataFrame({
                            'track_id': [track_id],
                            'track_name': [track_name],
                            'track_artist': [track_artist],
                            'acousticness': [audio_features_data['acousticness']],
                            'preview_url' : [preview_url]
                        })
                        
                        # Concatenate the new DataFrame with the existing DataFrame
                        acoustic_tracks_df = pd.concat([acoustic_tracks_df, track_df], ignore_index=True)
                        
                        total_acoustic = total_acoustic + 1
                        
                if preview_url and track_id not in non_acoustic_tracks_df['track_id'].values:
                    
                    # # Fetch track's audio features
                    # audio_features_url = f'https://api.spotify.com/v1/audio-features/{track_id}'
                    # audio_features_response = requests.get(audio_features_url, headers=headers)
                    # audio_features_data = audio_features_response.json()
                    
                    # Check if the track's acousticness is less than the threshold
                    if audio_features_data['acousticness'] < non_acousticness_threshold:
                        print(f'\t{track_name} - {track_artist} (Acousticness: {audio_features_data["acousticness"]})')
                        # Append the new track to the DataFrame
                        track_df = pd.DataFrame({
                            'track_id': [track_id],
                            'track_name': [track_name],
                            'track_artist': [track_artist],
                            'acousticness': [audio_features_data['acousticness']],
                            'preview_url' : [preview_url]
                        })
                        
                        # Concatenate the new DataFrame with the existing DataFrame
                        non_acoustic_tracks_df = pd.concat([non_acoustic_tracks_df, track_df], ignore_index=True)
                        
                        total_non_acoustic = total_non_acoustic + 1
                        
                if preview_url and track_id not in energetic_tracks_df['track_id'].values:
                    
                    # # Fetch track's audio features
                    # audio_features_url = f'https://api.spotify.com/v1/audio-features/{track_id}'
                    # audio_features_response = requests.get(audio_features_url, headers=headers)
                    # audio_features_data = audio_features_response.json()
                    
                    # Check if the track's energy is greater than the threshold
                    if audio_features_data['energy'] > energy_threshold:
                        print(f'\t{track_name} - {track_artist} (Energy: {audio_features_data["energy"]})')
                        # Append the new track to the DataFrame
                        track_df = pd.DataFrame({
                            'track_id': [track_id],
                            'track_name': [track_name],
                            'track_artist': [track_artist],
                            'energy': [audio_features_data['energy']],
                            'preview_url' : [preview_url]
                        })
                        
                        # Concatenate the new DataFrame with the existing DataFrame
                        energetic_tracks_df = pd.concat([energetic_tracks_df, track_df], ignore_index=True)
                        
                        total_energetic = total_energetic + 1
                        
                        
                if preview_url and track_id not in non_energetic_tracks_df['track_id'].values:
                    
                    # # Fetch track's audio features
                    # audio_features_url = f'https://api.spotify.com/v1/audio-features/{track_id}'
                    # audio_features_response = requests.get(audio_features_url, headers=headers)
                    # audio_features_data = audio_features_response.json()
                    
                    # Check if the track's energy is less than the threshold
                    if audio_features_data['energy'] < non_energy_threshold:
                        print(f'\t{track_name} - {track_artist} (Energy: {audio_features_data["energy"]})')
                        # Append the new track to the DataFrame
                        track_df = pd.DataFrame({
                            'track_id': [track_id],
                            'track_name': [track_name],
                            'track_artist': [track_artist],
                            'energy': [audio_features_data['energy']],
                            'preview_url' : [preview_url]
                        })
                        
                        # Concatenate the new DataFrame with the existing DataFrame
                        non_energetic_tracks_df = pd.concat([non_energetic_tracks_df, track_df], ignore_index=True)
                        
                        total_non_energetic = total_non_energetic + 1
                        
        else:
            print(f'Processing track {index+1} of {len(tracks_data)}')
            
            # add to all 
            track_df = pd.DataFrame({
                'track_id': [track_id],
                'track_name': [track_name],
                'track_artist': [track_artist],
                'danceability': [audio_features_data['danceability']],
                'instrumentalness': [audio_features_data['instrumentalness']],
                'speechiness': [audio_features_data['speechiness']],
                'acousticness': [audio_features_data['acousticness']],
                'energy': [audio_features_data['energy']],
                'valence': [audio_features_data['valence']],
                'tempo': [audio_features_data['tempo']],
                'loudness': [audio_features_data['loudness']],
                'liveness': [audio_features_data['liveness']],
                'key': [audio_features_data['key']],
                'mode': [audio_features_data['mode']],
                'time_signature': [audio_features_data['time_signature']],
                'preview_url' : [preview_url]
            })
            all_tracks_df = pd.concat([all_tracks_df, track_df], ignore_index=True)
            total = total + 1

    
            # Check if the track is already in the DataFrame
            if preview_url and track_id not in danceable_tracks_df['track_id'].values:
                # # Fetch track's audio features
                # audio_features_url = f'https://api.spotify.com/v1/audio-features/{track_id}'
                # audio_features_response = requests.get(audio_features_url, headers=headers)
                # audio_features_data = audio_features_response.json()

                # Check if the track's danceability is greater than the threshold
                if audio_features_data['danceability'] > danceability_threshold:
                    print(f'\t{track_name} - {track_artist} (Danceability: {audio_features_data["danceability"]})')
                    # Append the new track to the DataFrame
                    track_df = pd.DataFrame({
                        'track_id': [track_id],
                        'track_name': [track_name],
                        'track_artist': [track_artist],
                        'danceability': [audio_features_data['danceability']],
                        'preview_url' : [preview_url]
                    })
                    
                    # Concatenate the new DataFrame with the existing DataFrame
                    danceable_tracks_df = pd.concat([danceable_tracks_df, track_df], ignore_index=True)
                    
                    total_danceable = total_danceable + 1
                    
            # Check if the track is already in the DataFrame
            if preview_url and track_id not in non_danceable_tracks_df['track_id'].values:
            
                # # Fetch track's audio features
                # audio_features_url = f'https://api.spotify.com/v1/audio-features/{track_id}'
                # audio_features_response = requests.get(audio_features_url, headers=headers)
                # audio_features_data = audio_features_response.json()

                # Check if the track's danceability is less than the threshold
                if audio_features_data['danceability'] < non_danceability_threshold:
                    print(f'\t{track_name} - {track_artist} (Danceability: {audio_features_data["danceability"]})')
                    # Append the new track to the DataFrame
                    track_df = pd.DataFrame({
                        'track_id': [track_id],
                        'track_name': [track_name],
                        'track_artist': [track_artist],
                        'danceability': [audio_features_data['danceability']],
                        'preview_url' : [preview_url]
                    })
                    
                    # Concatenate the new DataFrame with the existing DataFrame
                    non_danceable_tracks_df = pd.concat([danceable_tracks_df, track_df], ignore_index=True)
                    
                    total_non_danceable = total_non_danceable + 1
                    
            
                
dataframes = {
    'Danceable Tracks': danceable_tracks_df,
    'Non-danceable Tracks': non_danceable_tracks_df,
    'Instrumental Tracks': instrumental_tracks_df,
    'Non-instrumental Tracks': non_instrumental_tracks_df,
    'Speechy Tracks': speechy_tracks_df,
    'Non-speechy Tracks': non_speechy_tracks_df,
    'Acoustic Tracks': acoustic_tracks_df,
    'Non-acoustic Tracks': non_acoustic_tracks_df,
    'Energetic Tracks': energetic_tracks_df,
    'Non-energetic Tracks': non_energetic_tracks_df,
    'All Tracks': all_tracks_df
}
           
print(f'Total tracks: {total}')
print(f'Total danceable tracks: {total_danceable}')
print(f'Total non-danceable tracks: {total_non_danceable}')
print(f'Total speechy tracks: {total_speechy}')
print(f'Total non-speechy tracks: {total_non_speechy}')
print(f'Total acoustic tracks: {total_acoustic}')
print(f'Total non-acoustic tracks: {total_non_acoustic}')
print(f'Total energetic tracks: {total_energetic}')
print(f'Total non-energetic tracks: {total_non_energetic}')
print(f'Total instrumental tracks: {total_instrumental}')
print(f'Total non-instrumental tracks: {total_non_instrumental}')

# # Display the DataFrame with danceable tracks
# print(danceable_tracks_df)

The above things all seem to have resulted in failure because of the lack of features for the collected tracks. Starting from scratch.

Save the dataframes

In [ ]:
def save_dataframes_to_csv(dataframes):
    for df_name, df in dataframes.items():
        filename = df_name.replace(" ", "_").lower() + "-2.csv"
        df.to_csv(filename, index=False)
        print(f"Saved {df_name} to {filename}")

save_dataframes_to_csv(dataframes)


In [ ]:
print(len(danceable_tracks_df))

#### New method of collecting audio based on purely gengres

In [51]:
!pip install tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 3.1 MB/s eta 0:00:00


In [49]:
import time
import random

In [58]:
def search_tracks(query, limit, offset):
    results = sp.search(q=query, type='track', limit=limit, offset=offset)
    return results['tracks']['items']

def get_track_features(track_ids):
    features = sp.audio_features(track_ids)
    return features


In [75]:
import requests
import time

MAX_RETRIES = 3
RETRY_DELAY = 5  # seconds

def search_tracks(query, limit, offset):
    retries = 0
    while retries < MAX_RETRIES:
        try:
            # response = requests.get(
            #     'https://api.spotify.com/v1/search',
            #     params={
            #         'q': query,
            #         'type': 'track',
            #         'limit': limit,
            #         'offset': offset
            #     }
            # )
            response = sp.search(q=query, type='track', limit=limit, offset=offset)
            # response.raise_for_status()
            #data = response.json()
            return response['tracks']['items']
        except (requests.exceptions.RequestException, ValueError):
            # Catch any HTTP errors or JSON parsing errors
            retries += 1
            time.sleep(RETRY_DELAY)
    # If we've exceeded the maximum number of retries, raise an exception
    raise Exception(f"Failed to fetch search results after {MAX_RETRIES} retries.")


In [79]:
import random

required_features = ['danceability', 'instrumentalness', 'speechiness', 'acousticness', 'energy', 'valence', 'tempo', 'loudness', 'liveness', 'key', 'mode', 'time_signature']

def compile_tracks(query, total_tracks, limit=50):
    tracks_df = pd.DataFrame()
    keywords = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

    # Initialize the progress bar
    with tqdm(total=total_tracks) as pbar:
        while tracks_df.shape[0] < total_tracks:
            keyword = random.choice(keywords)
            remaining_tracks = total_tracks - tracks_df.shape[0]
            current_limit = min(limit, remaining_tracks)
            offset = 0

            while offset < 800:
                search_result = search_tracks(f"{query} {keyword}", current_limit, offset)
                track_ids = [track['id'] for track in search_result]
                track_previews = [track['preview_url'] for track in search_result]
                track_names = [track['name'] for track in search_result]

                if not track_ids:
                    break

                features = get_track_features(track_ids)

                temp_df = pd.DataFrame(features)
                temp_df['preview_url'] = track_previews
                temp_df['track_name'] = track_names

                # Filter out tracks that do not have all the required features
                temp_df = temp_df.dropna(subset=required_features)

                # Filter out the duplicate tracks if the 'id' column exists in both DataFrames
                if 'id' in tracks_df.columns and 'id' in temp_df.columns:
                    temp_df = temp_df.loc[~temp_df['id'].isin(tracks_df['id'])]

                tracks_df = pd.concat([tracks_df, temp_df], ignore_index=True)

                # Update the progress bar with the number of new tracks added
                pbar.update(temp_df.shape[0])

                offset += current_limit
                time.sleep(random.uniform(1, 3))

            if tracks_df.shape[0] >= total_tracks:
                break

    return tracks_df


In [80]:
genres = ['pop', 'rock', 'hip-hop', 'classical', 'jazz', 'country', 'electronic', 'reggae']
total_tracks_per_genre = 20000 // len(genres)
tracks_df_full = pd.DataFrame()

for genre in genres:
    query = f"genre:{genre}"
    genre_tracks_df = compile_tracks(query, total_tracks_per_genre)
    # tracks_df = tracks_df.append(genre_tracks_df, ignore_index=True)
    tracks_df_full = pd.concat([tracks_df_full, genre_tracks_df], ignore_index=True)
    print(len(tracks_df_full))

tracks_df_full.to_csv('tracks_features.csv', index=False)


 12%|█▏        | 300/2500 [00:21<02:22, 15.43it/s]

In [78]:
print(len(track_df_full))

1
